In [1]:
import os
import pandas as pd

import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from aux_functions import sqllite_query
sqllite_db = 'SQLITE/sqlite3.db'

conn = sqlite3.connect(sqllite_db, timeout = 5)
engine = sqlalchemy.create_engine(f'sqlite:///{sqllite_db}', echo=False)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [ ]:
# SHOW TABLES
sql = "SELECT * FROM sqlite_master WHERE type='table'"
sqllite_query(sqllite_db, sql, bln_read_results_to_df = True)
#df.head(10)

In [ ]:
# SHOW TABLES
sql = "SELECT * FROM previous_application limit 1"
df = sqllite_query(sqllite_db, sql, bln_read_results_to_df = True)
df.T

# PREVIOUS APPLICATION

In [6]:
sql = """

with apps as (

select
  t.sk_id_curr
, t.sk_id_prev  
, row_number() over (partition by t.sk_id_curr order by t.days_decision desc) as rn_last_prev_app
, row_number() over (partition by t.sk_id_curr order by t.days_decision) as rn_first_prev_app
from previous_application t

)

, loans as (

select
  t.sk_id_curr
, t.sk_id_prev  
, row_number() over (partition by t.sk_id_curr order by t.days_decision desc) as rn_last_prev_loan
, row_number() over (partition by t.sk_id_curr order by t.days_decision) as rn_first_prev_loan

from previous_application t
where t.name_contract_status = 'Approved'
)


SELECT 
  t.sk_id_curr

-- total
, count(1) as prev_cnt_apps
, count(case when t.name_contract_type = 'Consumer loans' then 1 end) as prev_cnt_consumer_apps
, count(case when t.name_contract_type = 'Cash loans' then 1 end) as prev_cnt_cash_apps
, count(case when t.name_contract_type = 'Revolving loans' then 1 end) as prev_cnt_revolving_apps

-- loans only
, count(case when t.name_contract_status = 'Approved' then 1 end) as prev_cnt_loans
, count(case when t.name_contract_status = 'Approved' and t.name_contract_type = 'Consumer loans' then 1 end) as prev_cnt_consumer_loans
, count(case when t.name_contract_status = 'Approved' and t.name_contract_type = 'Cash loans' then 1 end) as prev_cnt_cash_loans
, count(case when t.name_contract_status = 'Approved' and t.name_contract_type = 'Revolving loans' then 1 end) as prev_cnt_revolving_loans
  
-- last application  
, max(case when a.rn_last_prev_app = 1 then t.amt_credit else 0 end) as prev_last_app_amt_credit
, max(case when a.rn_last_prev_app = 1 then t.name_contract_status end) as prev_last_app_name_contract_status
, max(case when a.rn_last_prev_app = 1 then t.days_decision else 0 end) as prev_last_app_days_decision
, max(case when a.rn_last_prev_app = 1 then t.name_client_type end) as prev_last_app_name_client_type
, max(case when a.rn_last_prev_app = 1 then t.name_contract_type end) as prev_last_app_name_contract_type
, max(case when a.rn_last_prev_app = 1 then t.product_combination end) as prev_last_app_product_combination

-- first application
, max(case when a.rn_first_prev_app = 1 then t.amt_credit else 0 end) as prev_first_app_amt_credit
, max(case when a.rn_first_prev_app = 1 then t.name_contract_status end) as prev_first_app_name_contract_status
, max(case when a.rn_first_prev_app = 1 then t.days_decision else 0 end) as prev_first_app_days_decision
, max(case when a.rn_first_prev_app = 1 then t.name_client_type end) as prev_first_app_name_client_type
, max(case when a.rn_first_prev_app = 1 then t.name_contract_type end) as prev_first_app_name_contract_type
, max(case when a.rn_first_prev_app = 1 then t.product_combination end) as prev_first_app_product_combination

-- last loan
, max(case when l.rn_last_prev_loan = 1 then t.amt_credit else 0 end) as prev_last_loan_amt_credit
, max(case when l.rn_last_prev_loan = 1 then t.days_decision end) as prev_last_loan_days_decision
, max(case when l.rn_last_prev_loan = 1 then t.days_first_due end) as prev_last_loan_days_first_due
, max(case when l.rn_last_prev_loan = 1 then t.days_last_due end) as prev_last_loan_days_last_due
, max(case when l.rn_last_prev_loan = 1 then t.days_termination end) as prev_last_loan_days_termination
, max(case when l.rn_last_prev_loan = 1 then i.rate_interest end) as prev_last_loan_rate_interest

-- first loan
, max(case when l.rn_first_prev_loan = 1 then t.amt_credit else 0 end) as prev_first_loan_amt_credit
, max(case when l.rn_first_prev_loan = 1 then t.days_decision end) as prev_first_loan_days_decision
, max(case when l.rn_first_prev_loan = 1 then t.days_first_due end) as prev_first_loan_days_first_due
, max(case when l.rn_first_prev_loan = 1 then t.days_first_due end) as prev_first_loan_days_first_due
, max(case when l.rn_first_prev_loan = 1 then t.days_termination end) as prev_first_loan_days_termination
, max(case when l.rn_first_prev_loan = 1 then i.rate_interest end) as prev_first_loan_rate_interest


-- max vs min
, max(t.amt_credit) - min(t.amt_credit) as prev_max_min_amt_credit
, max(t.days_decision) - min(t.days_decision) as prev_max_min_days_decision
, max(i.rate_interest) - min(i.rate_interest) as prev_max_min_rate_interest

from previous_application t
join prev_app_irr i on t.sk_id_curr = i.sk_id_curr and t.sk_id_prev = i.sk_id_prev
join apps a on t.sk_id_curr = a.sk_id_curr and t.sk_id_prev = a.sk_id_prev
left join loans l on t.sk_id_curr = l.sk_id_curr and t.sk_id_prev = l.sk_id_prev

group by t.sk_id_curr

"""
df = sqllite_query(sqllite_db, sql, bln_read_results_to_df = True)

df.head(3)

,SK_ID_CURR,prev_cnt_apps,prev_cnt_consumer_apps,prev_cnt_cash_apps,prev_cnt_revolving_apps,prev_cnt_loans,prev_cnt_consumer_loans,prev_cnt_cash_loans,prev_cnt_revolving_loans,prev_last_app_amt_credit,prev_last_app_name_contract_status,prev_last_app_days_decision,prev_last_app_name_client_type,prev_last_app_name_contract_type,prev_last_app_product_combination,prev_first_app_amt_credit,prev_first_app_name_contract_status,prev_first_app_days_decision,prev_first_app_name_client_type,prev_first_app_name_contract_type,prev_first_app_product_combination,prev_last_loan_amt_credit,prev_last_loan_days_decision,prev_last_loan_days_first_due,prev_last_loan_days_last_due,prev_last_loan_days_termination,prev_last_loan_rate_interest,prev_first_loan_amt_credit,prev_first_loan_days_decision,prev_first_loan_days_first_due,prev_first_loan_days_first_due,prev_first_loan_days_termination,prev_first_loan_rate_interest,prev_max_min_amt_credit,prev_max_min_days_decision,prev_max_min_rate_interest
0,100001,1,1,0,0,1,1,0,0,23787.0,Approved,-1740,Refreshed,Consumer loans,POS mobile with interest,23787.0,Approved,-1740,Refreshed,Consumer loans,POS mobile with interest,23787.0,-1740.0,-1709.0,-1619.0,-1612.0,0.814646,23787.0,-1740.0,-1709.0,-1709.0,-1612.0,0.814646,0.0,0,0.00000
1,100002,1,1,0,0,1,1,0,0,179055.0,Approved,-606,New,Consumer loans,POS other with interest,179055.0,Approved,-606,New,Consumer loans,POS other with interest,179055.0,-606.0,-565.0,-25.0,-17.0,0.215780,179055.0,-606.0,-565.0,-565.0,-17.0,0.215780,0.0,0,0.00000
2,100003,3,2,1,0,3,2,1,0,1035882.0,Approved,0,Repeater,Cash loans,Cash X-Sell: low,68053.5,Approved,0,Refreshed,Consumer loans,POS household with interest,1035882.0,-746.0,-716.0,-536.0,-527.0,0.248058,68053.5,-2341.0,-2310.0,-2310.0,-1976.0,0.330635,967828.5,1595,0.12375


In [7]:
df.to_sql('features_prev_app_agg', con=engine, if_exists='replace', index = False) # append

338857

# BUREAU

In [8]:
sql = """

SELECT 
  t.sk_id_curr
  
, count(1) as bur_cnt_credit
, count(case when t.credit_active = 'Closed' then 1 end) as bur_cnt_credit_closed
, count(case when t.credit_active = 'Active' then 1 end) as bur_cnt_credit_active
, count(case when t.credit_day_overdue > 0 then 1 end) as bur_cnt_credit_overdue

, max(t.days_credit) as bur_max_days_credit
, min(t.days_credit) as bur_min_days_credit
, max(t.days_credit_enddate) as bur_max_days_credit_enddate
, min(t.days_credit_enddate) as bur_min_days_credit_enddate

, sum(t.amt_credit_sum_debt) as bur_sum_amt_credit_sum_debt
, min(t.amt_credit_sum_debt) as bur_min_amt_credit_sum_debt
, max(t.amt_credit_sum_debt) as bur_max_amt_credit_sum_debt
, max(t.amt_credit_sum_debt) - min(t.amt_credit_sum_debt) as bur_max_min_amt_credit_sum_debt

, count(case when t.DAYS_CREDIT > -30 then 1 end) as bur_cnt_credit_30
, count(case when t.DAYS_CREDIT > -90 then 1 end) as bur_cnt_credit_90
, count(case when t.DAYS_CREDIT > -180 then 1 end) as bur_cnt_credit_180
, count(case when t.DAYS_CREDIT > -360 then 1 end) as bur_cnt_credit_360

from bureau t

group by t.sk_id_curr


"""
df = sqllite_query(sqllite_db, sql, bln_read_results_to_df = True)

df.head(3)

,SK_ID_CURR,bur_cnt_credit,bur_cnt_credit_closed,bur_cnt_credit_active,bur_cnt_credit_overdue,bur_max_days_credit,bur_min_days_credit,bur_max_days_credit_enddate,bur_min_days_credit_enddate,bur_sum_amt_credit_sum_debt,bur_min_amt_credit_sum_debt,bur_max_amt_credit_sum_debt,bur_max_min_amt_credit_sum_debt,bur_cnt_credit_30,bur_cnt_credit_90,bur_cnt_credit_180,bur_cnt_credit_360
0,100001,7,4,3,0,-49,-1572,1778.0,-1329.0,596686.5,0.0,373239.0,373239.0,0,1,1,2
1,100002,8,6,2,0,-103,-1437,780.0,-1072.0,245781.0,0.0,245781.0,245781.0,0,0,1,1
2,100003,4,3,1,0,-606,-2586,1216.0,-2434.0,0.0,0.0,0.0,0.0,0,0,0,0


In [9]:
df.to_sql('features_bureau_agg', con=engine, if_exists='replace', index = False) # append

305811